In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#PDFs in BDT and sindec?
import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
#from icecube import astro
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)


#import seaborn as sns
#plt.rc('font', size=14)


## Define Settings

In [ ]:
selection_version = 'version-001-p00'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    data_prefix = '/data/user/ssclafani/data/cscd/final'
    ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = '/data/user/mhuennefeld/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/gp_flux_comparison'
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

#### Define Colors

In [ ]:
colors = ['#0B3D53', '#FF7F0E',  '#5C9FC9']
colors

#### Load Templates

In [ ]:
kra5, _, _, ebins5 = np.load('/data/ana/analyses/NuSources/2021_DNNCascade_analyses/templates/KRA-gamma_5PeV_maps_energies.tuple.npy', allow_pickle = True, encoding='latin1')
kra50, _, _, ebins50 = np.load('/data/ana/analyses/NuSources/2021_DNNCascade_analyses/templates/KRA-gamma_maps_energies.tuple.npy', allow_pickle=True, encoding='latin1')
pi0 = np.load('/data/ana/analyses/NuSources/2021_DNNCascade_analyses/templates/Fermi-LAT_pi0_map.npy', allow_pickle=True, encoding='latin1')


In [ ]:
import healpy as hp
a = hp.nside2pixarea(128)
print(a)

In [ ]:
dat = np.loadtxt('../paper/saved_sens/piecewise_fitresult.txt', skiprows = 13)
x = dat[:,0]
x_err_low = x - dat[:,1]
x_err_high = dat[:,2] - x

# from fit
tnorm = 1.0e-8 * (4*np.pi)

y = tnorm * dat[:,3]

y_max = tnorm * dat[:,-1]
y_min = tnorm * dat[:,-2]

y_err_high = y_max - y
y_err_low = y - y_min

indices_UL = np.array([5,8,9, 10, 11, 12])

# visualize upper limits with corresponding marker
y[indices_UL] = y[indices_UL]+y_err_high[indices_UL]

uplims = np.zeros(x.shape)
uplims[[indices_UL]]=True

# plot the points with non-zero best fit (i.e. not upper-limits)
x_new = np.delete(x, indices_UL)
x_err_high_new = np.delete(x_err_high, indices_UL)
x_err_low_new = np.delete(x_err_low, indices_UL)

y_new = np.delete(y, indices_UL)
y_err_high_new = np.delete(y_err_high, indices_UL)
y_err_low_new = np.delete(y_err_low, indices_UL)

In [ ]:
georg_flux = np.load('/data/user/gschwefer/misc/for_steve/fermi_pi0_gamma_ray_allsky_average_E2Phi_2.7_extrapol.npy', allow_pickle=True)
georg_energy = np.load('/data/user/gschwefer/misc/for_steve/fermi_pi0_gamma_ray_energies.npy', allow_pickle=True)
georg_flux_gamma = np.array(georg_flux)
georg_flux = 0.31* georg_flux # this is to convert gamma


In [ ]:
hans_result = np.loadtxt('../paper/saved_sens/Cascades_PRL_2010_11_12_13_14_15_pp_nominal_1sigma.txt', skiprows=1)

In [ ]:
hans_gammas = hans_result[:,0]
hans_norm = hans_result[:,1]

In [ ]:
hans_flux = {}
for i, gamma in enumerate(hans_gammas):
    Es = np.linspace(16000,2.6e6, 1000)
    hans_flux[gamma] =  Es**2*1e-18*hans_norm[i]*((Es/1e5)**(-1*hans_gammas[i]))
    
bestfithans = Es**2*1e-18*1.66*((Es/1e5)**-2.53)

In [ ]:
np.log10(16000)

In [ ]:
points = np.ndarray((len(hans_gammas), 1000))
xs = np.logspace(4.20411998,6.41497334,1000)

for i, gamma in enumerate(hans_gammas):
    points[i] = np.interp(np.log10(xs) ,np.log10(Es), hans_flux[gamma])
    

In [ ]:
print(points.shape)

mins = np.min(points, axis=0)
maxs = np.max(points, axis=0)

In [ ]:
#for gamma in (hans_gammas):
#    plt.plot(Es, hans_flux[gamma], c='grey')
#for i, gamma in enumerate(hans_gammas):
#    plt.plot(xs, points[i])
plt.fill_between(xs, mins, maxs, color='r', alpha=0.25)
plt.plot(Es, bestfithans, c='r', ls='--')
plt.loglog()
plt.ylim(5e-10,1e-6)
#plt.xlim(1e4,1e8)

In [ ]:
''' pi0flux = [1271.8527801661924, 0.020005181457912437,
1553.1845690893892, 0.01852439272816013,
1922.519128617942, 0.016478593938396482,
2381.49216131387, 0.014615905579883785,
2895.9554838722925, 0.012866607693166323,
3585.370348580028, 0.011101706303158538,
4375.590098581272, 0.009635108280540587,
5397.82609630125, 0.008285718315429003,
6645.846422152243, 0.007095591309854342,
8182.418973649131, 0.006067527085469145,
10074.25932040286, 0.005203618510847123,
12318.748238749182, 0.004495475813262984,
15166.940810483744, 0.0038723560749193814,
19149.109652203424, 0.0032775651321566347,
23576.536108990374, 0.0028150126763895286,
28772.831793568785, 0.0024502918550690176,
35494.8107159693, 0.002113304010356377,
43701.49326658316, 0.001825711674448281,
53717.853633984676, 0.0015706784002100286,
65793.2201885699, 0.0013546635088702511,
81005.13599200615, 0.0011600851314746696,
94194.95560159562, 0.001041064937747737
]
pi0_EGeV = np.array(pi0flux[::2]) /1e3
pi0_fluxGeV = np.array(pi0flux[1::2])  /1e3    
b1, b0, _r, _p_val, _stderr = scipy.stats.linregress(np.log10(pi0_EGeV[11:]), 
                                           np.log10(pi0_fluxGeV[11:])) 
x_pred = np.linspace(2,7, 100)
y_pred = b1 * x_pred + b0   '''

In [ ]:
erange_kra5 = [5000, 160000]
erange_kra50 = [6500, 315000]
erange_pi0 = [1800, 316000]

#ranges from ts
#erange_kra5 = [800, 100000]
#erange_kra50 = [1000, 146000]
#erange_pi0 = [800, 69000]

In [ ]:
mask5 = (ebins5 > erange_kra5[0]) & (ebins5 < erange_kra5[1])
mask50 = (ebins50 > erange_kra50[0]) & (ebins50 < erange_kra50[1])
#maskpi0 = (10**x_pred > 1500) & (10**x_pred < 315000)
maskpi0_georg = (georg_energy > np.log10(erange_pi0[0])) & (georg_energy < np.log10(erange_pi0[1]))

In [ ]:
pi0modelnorm = 4.43e-12
pi0_range = [1.7e-11, 2.18e-11, 2.69e-11]
pi0_range = np.array(pi0_range) / pi0modelnorm
print(pi0_range)
print(pi0_range[0] - pi0_range[1])
print(pi0_range[2] - pi0_range[1])

In [ ]:
#pi0_range = [4.23, 5.45, 7]
kra5_range = [.659, .893,  1.13]
kra50_range = [.423, .591, 0.768]

In [ ]:
print(kra5_range[0] - kra5_range[1])
print(kra5_range[2] - kra5_range[1])

In [ ]:
print(kra50_range[0] - kra50_range[1])
print(kra50_range[2] - kra50_range[1])

In [ ]:
fig , ax = plt.subplots(figsize=(5.5,5.5))
plt.rc('font', size=10)
ax.plot(ebins5, ebins5**2 * (sum(kra5)) * hp.nside2pixarea(128) / 3 , ls=':', color=colors[0] ,
        lw=2, alpha=0.8, label='KRA$_\gamma^{5}$ Model')
ax.plot(ebins50, ebins50**2 * (sum(kra50)) * hp.nside2pixarea(128)/ 3, ls=':', color=colors[2],
        lw=2, alpha=0.8, label='KRA$_\gamma^{50}$ Model')
ax.plot(10**georg_energy, 4*np.pi*georg_flux, color=colors[1],
        lw=2, alpha=0.8, ls=':', label='$\pi^0$ Model')

ax.plot(np.nan, np.nan, label=' ', alpha=0)


#ax.plot(10 ** x_pred, 4*np.pi*10 ** y_pred, lw=3, linestyle = ":", 
#           label='$\pi^0$ Model')
'''(_, caps, _) = ax.errorbar(x[indices_UL], y[indices_UL], 
                           xerr=[x_err_low[indices_UL], x_err_high[indices_UL]], yerr=y[indices_UL]*0.5, fmt='o', 
                           c='dimgrey', linewidth=2.0, zorder=30, uplims=True, markersize=0)
(_, caps2, _) = ax.errorbar(x_new, y_new, xerr=[x_err_low_new, x_err_high_new], 
                            yerr=[y_err_low_new, y_err_high_new], fmt='o', c ='dimgrey', label="All-Sky Measured Flux", linewidth=2.0, zorder=31, markersize=0)
(_, caps4, _) = ax.errorbar(x_new, y_new, yerr=[y_err_low_new, y_err_high_new], 
                            fmt='o', c='dimgrey' , linewidth=2.0, zorder=31, markersize=0, capsize=0)

for cap in caps:
    cap.set_markeredgewidth(2)
'''
kra5_plot = ebins5[mask5]**2 *(sum(kra5)[mask5]) * hp.nside2pixarea(128)/ 3

ax.plot(ebins5[mask5], kra5_range[1] * kra5_plot , 
         c =colors[0], ls='-', zorder=10, lw=2, label='KRA$_\gamma^{5}$ Best-Fit Flux')
ax.fill_between(ebins5[mask5], kra5_range[0] * kra5_plot , kra5_range[-1] * kra5_plot,
        color = colors[0], alpha=.25, zorder=10)

kra50_plot = ebins50[mask50]**2 *(sum(kra50)[mask50]) * hp.nside2pixarea(128) / 3
ax.plot(ebins50[mask50], kra50_range[1] * kra50_plot,
         c =colors[2], ls='-', zorder=5, lw=2, label='KRA$_\gamma^{50}$ Best-Fit Flux')
ax.fill_between(ebins50[mask50],kra50_range[0] * kra50_plot, kra50_range[-1] * kra50_plot ,
        color = colors[2], zorder=5,  alpha=.25)

ax.plot(10**georg_energy[maskpi0_georg], pi0_range[1]*4*np.pi*georg_flux[maskpi0_georg], 
        lw=2, c=colors[1], ls='-', zorder=3,
        label='$\pi^0$ Best-Fit Flux')
ax.fill_between(10**georg_energy[maskpi0_georg], pi0_range[0]*4*np.pi*georg_flux[maskpi0_georg],
                pi0_range[-1]*4*np.pi*georg_flux[maskpi0_georg], zorder=3, color=colors[1], alpha=.25 )


#plt.plot(Es, 4*np.pi*bestfithans, c='dimgrey', lw='1', alpha=0.4, ls='--', label='M.G.Aartsen(2020)')
plt.fill_between(xs, 4*np.pi*mins, 4*np.pi*maxs, color='dimgrey', alpha=0.1,
                 edgecolor='k', label=r'M.G.$\,$Aartsen et al.(2020)')

plt.loglog()
#ax.set_ylim(1e-12,5e-9)
ax.set_ylim(5e-9,5e-6)
ax.set_xlim(500, 1e7)
plt.legend(ncol=2)
plt.rc('font', size=13)
ax.set_ylabel(r'E$_{\nu}^2$ $\frac{dN}{dE_\nu}$ [GeV s$^{-1}$ cm$^{-2}$]')
ax.set_xlabel(r'E$_\nu$ [GeV]')
plt.tight_layout()
fig.savefig('{}/gp_flux.png'.format(plot_dir))


In [ ]:
fig , ax = plt.subplots(figsize=(5.5,5.5))
plt.rc('font', size=10)
ax.plot(ebins5, ebins5**2 * (sum(kra5)) * hp.nside2pixarea(128) / (4*np.pi) , ls=':', color=colors[0] ,
        lw=2, alpha=0.8, label='KRA$_\gamma^{5}$ Model')
ax.plot(ebins50, ebins50**2 * (sum(kra50)) * hp.nside2pixarea(128)/ (4*np.pi), ls=':', color=colors[2],
        lw=2, alpha=0.8, label='KRA$_\gamma^{50}$ Model')
ax.plot(10**georg_energy, 3*georg_flux, color=colors[1],
        lw=2, alpha=0.8, ls=':', label='$\pi^0$ Model')

ax.plot(np.nan, np.nan, label=' ', alpha=0)

kra5_plot = ebins5[mask5]**2 *(sum(kra5)[mask5]) * hp.nside2pixarea(128)/ (4*np.pi)

ax.plot(ebins5[mask5], kra5_range[1] * kra5_plot , 
         c =colors[0], ls='-', zorder=10, lw=2, label='KRA$_\gamma^{5}$ Best-Fit Flux')
ax.fill_between(ebins5[mask5], kra5_range[0] * kra5_plot , kra5_range[-1] * kra5_plot,
        color = colors[0], alpha=.25, zorder=10)

kra50_plot = ebins50[mask50]**2 *(sum(kra50)[mask50]) * hp.nside2pixarea(128) / (4*np.pi)
ax.plot(ebins50[mask50], kra50_range[1] * kra50_plot,
         c =colors[2], ls='-', zorder=5, lw=2, label='KRA$_\gamma^{50}$ Best-Fit Flux')
ax.fill_between(ebins50[mask50],kra50_range[0] * kra50_plot, kra50_range[-1] * kra50_plot ,
        color = colors[2], zorder=5,  alpha=.25)

ax.plot(10**georg_energy[maskpi0_georg], 3*pi0_range[1]*georg_flux[maskpi0_georg], 
        lw=2, c=colors[1], ls='-', zorder=3,
        label='$\pi^0$ Best-Fit Flux')
ax.fill_between(10**georg_energy[maskpi0_georg], 3*pi0_range[0]*georg_flux[maskpi0_georg],
                3*pi0_range[-1]*georg_flux[maskpi0_georg], zorder=3, color=colors[1], alpha=.25 )


#plt.plot(Es, 4*np.pi*bestfithans, c='dimgrey', lw='1', alpha=0.4, ls='--', label='M.G.Aartsen(2020)')
plt.fill_between(xs, 3*mins,3* maxs, color='dimgrey', alpha=0.1,
                 edgecolor='k', label=r'M.G.$\,$Aartsen et al.(2020)')

plt.loglog()
#ax.set_ylim(1e-12,5e-9)
ax.set_ylim(1e-9,1.5e-6)
ax.set_xlim(500, 1e7)
plt.legend(ncol=2)
plt.rc('font', size=13)
ax.set_ylabel(r'E$_{6\nu}^2$ $\frac{dN}{dE_\nu}$ [GeV s$^{-1}$ cm$^{-2}$ sr$^{-1}$]')
ax.set_xlabel('E [GeV]')
plt.tight_layout()
fig.savefig('{}/gp_flux_sr.png'.format(plot_dir))


## Make Flux Comparison Plots for different Sky Regions

#### Helper Functions

In [ ]:
def plot_skymap(skymap, fig=None, ax=None, figsize=(9, 6), **kwargs):
    if ax is None:
        fig, ax = plt.subplots(
            subplot_kw=dict(projection='aitoff'), figsize=figsize)
    sp = cy.plotting.SkyPlotter()
    pc, cb = sp.plot_map(ax, skymap)
    return fig, ax


"""
def equatorial_to_galactic(m, rot=180):
    r = hp.Rotator(rot=rot, coord='CG')
    return r.rotate_map_pixel(m)

def equatorial_to_galactic_coords(theta, phi, rot=180):
    r = hp.Rotator(rot=rot, coord='CG')
    return r(theta, phi)

def ra_dec_to_lon_lat(ra, dec):
    r = hp.Rotator(rot=rot, coord='CG')
    return r(theta, phi)
"""

def template_to_galactic_coords(theta, phi):
    """Convert template coordinates (equatorial coordinates, 
    but defined in theta [0, pi] instead of dec [-pi/2, pi/2]) 
    to galactic coordinates, i.e. Galactic longitude l and
    Galactic latitude b.
    """

    theta = np.atleast_1d(theta)
    phi = np.atleast_1d(phi)

    r = hp.Rotator(rot=0, coord='CG')
    theta_gal, phi_gal = r(theta, phi)
    lat = theta_gal - np.pi/2.
    lon = phi_gal
    if isinstance(lon, float):
        if lon < 0:
            lon = lon + 2*np.pi
    else:
        lon[lon < 0] += 2*np.pi
    return lon, lat

def get_pixel_mask(lon_min_deg, lon_max_deg, lat_min_deg=-5, lat_max_deg=5, nside=128):

    lon_min = np.deg2rad(lon_min_deg)
    lon_max = np.deg2rad(lon_max_deg)
    lat_min = np.deg2rad(lat_min_deg)
    lat_max = np.deg2rad(lat_max_deg)

    npix = hp.nside2npix(nside)
    theta, phi = hp.pix2ang(nside, np.r_[:npix])

    lon, lat = template_to_galactic_coords(theta, phi)

    mask = np.logical_and(lon > lon_min, lon < lon_max)
    mask = np.logical_and(mask, lat > lat_min)
    mask = np.logical_and(mask, lat < lat_max)
    return mask


In [ ]:
# ----------------------
# Run some sanity checks
# ----------------------
for cuts in [(25, 100), (50, 200)]:
    pi0_masked = np.array(pi0)
    mask = get_pixel_mask(*cuts)
    print('mask', cuts, np.sum(mask))
    pi0_masked[~mask] = np.inf
    fig, ax = plot_skymap(pi0_masked)

max_ipix = np.argmax(pi0)
max_dir = np.rad2deg(hp.pix2ang(nside=hp.get_nside(pi0), ipix=max_ipix))
print('max', max_ipix, max_dir, np.rad2deg(template_to_galactic_coords(*np.deg2rad(max_dir))))
#plot_skymap(equatorial_to_galactic(pi0))
np.rad2deg(template_to_galactic_coords(theta=np.pi/2. - np.deg2rad(22), phi=np.deg2rad(83.63)))


#### Load Tibet Data

In [ ]:
import pandas as pd

df_tibet_points = pd.read_csv('data/plota_TibetData.csv')
df_tibet_points_b = pd.read_csv('data/plotb_TibetData.csv')
df_tibet_theory = pd.read_csv('data/plota_theory.csv')

def convert_E27_to_E20(energies, E27dNdE):
    E20dNdE = E27dNdE / (energies**0.7)
    return E20dNdE

# convert Tibet data points from E^-2.7 to E^-2
df_tibet_points_E2 = {}
df_tibet_points_E2b = {}
for key in ['TibetBestFit', 'TibetYupper', 'TibetYLower', 'TibetXLower', 'TibetXUpper']:
    energies_gev = df_tibet_points[key + '_x'] * 1000.
    df_tibet_points_E2[key + '_x'] = energies_gev
    df_tibet_points_E2[key + '_y'] = convert_E27_to_E20(
        energies=energies_gev, E27dNdE=df_tibet_points[key + '_y'])

    energies_gev = df_tibet_points_b[key + '_x'] * 1000.
    df_tibet_points_E2b[key + '_x'] = energies_gev
    df_tibet_points_E2b[key + '_y'] = convert_E27_to_E20(
        energies=energies_gev, E27dNdE=df_tibet_points_b[key + '_y'])

df_tibet_points_E2 = pd.DataFrame(df_tibet_points_E2)
df_tibet_points_E2b = pd.DataFrame(df_tibet_points_E2b)

# convert Tibet theory points from E^-2.7 to E^-2
df_tibet_theory_E2 = {}
for key in ['SpaceIndependentCR', 'SpaceDependentCR']:
    energies_gev = df_tibet_theory[key + '_x'] * 1000.
    df_tibet_theory_E2[key + '_x'] = energies_gev
    df_tibet_theory_E2[key + '_y'] = convert_E27_to_E20(
        energies=energies_gev, E27dNdE=df_tibet_theory[key + '_y'])

df_tibet_theory_E2 = pd.DataFrame(df_tibet_theory_E2)

df_tibet_points_E2


#### Make Plots

In [ ]:
 for lon_min_deg, lon_max_deg, lat_deg in [(25, 100, 5), (50, 200, 5), (-45, 45, 5), (0, 360, 90)]:

    # get healpix mask for region
    mask_area = get_pixel_mask(lon_min_deg=lon_min_deg, lon_max_deg=lon_max_deg, lat_min_deg=-lat_deg, lat_max_deg=lat_deg)

    # compute area of masked sky in steradian
    sky_area = hp.nside2pixarea(128) * np.sum(mask_area)

    # compute total flux per energy bin for KRA-5 model
    kra5_flux = sum(kra5[mask_area])
    kra50_flux = sum(kra50[mask_area])
    print('kra5 flux ratio:', np.sum(kra5_flux) / np.sum(kra5))
    print('kra50 flux ratio:', np.sum(kra50_flux) / np.sum(kra50))

    # compute fraction of pi0 flux that's located in this sky region
    pi0_flux_ratio = np.sum(pi0[mask_area]) / np.sum(pi0)
    print('Pi0 flux ratio:', pi0_flux_ratio)

    fig, ax = plt.subplots(figsize=(9, 6))

    if (lon_min_deg, lon_max_deg, lat_deg) == (25, 100, 5):
        df_tibet = df_tibet_points_E2
    elif (lon_min_deg, lon_max_deg, lat_deg) == (50, 200, 5):
        df_tibet = df_tibet_points_E2b
    else:
        df_tibet = None

    if df_tibet is not None:
        #ax.scatter(df_tibet_points_E2['TibetBestFit_x'], df_tibet_points_E2['TibetBestFit_y'] * 0.31, label=r'Tibet AS+MD [$E_\gamma$ * 0.31]', color='red')
        xerr = [
            (df_tibet['TibetBestFit_x'] - df_tibet['TibetXLower_x']) * 2,
            (df_tibet['TibetXUpper_x'] - df_tibet['TibetBestFit_x']) * 2,
        ]
        yerr = [
            (df_tibet['TibetBestFit_y'] - df_tibet['TibetYLower_y']) * 0.31,
            (df_tibet['TibetYupper_y'] - df_tibet['TibetBestFit_y']) * 0.31,
        ]
        ax.errorbar(
            df_tibet['TibetBestFit_x'] * 2,
            df_tibet['TibetBestFit_y'] * 0.31,
            xerr=xerr,
            yerr=yerr,
            label=r'Tibet AS+MD [$E_\gamma$ * 2, $dN/dE_\gamma$ * 0.31]', color='red', fmt='.',
        )
        #ax.plot(df_tibet_theory_E2['SpaceIndependentCR_x'], df_tibet_theory_E2['SpaceIndependentCR_y'], label='Space Independent CR')
        #ax.plot(df_tibet_theory_E2['SpaceDependentCR_x'], df_tibet_theory_E2['SpaceDependentCR_y'], label='Space Independent CR')



    # plot measurements
    if True:
        kra5_plot = ebins5[mask5]**2 *(kra5_flux[mask5]) * hp.nside2pixarea(128) / 3 / (sky_area)

        ax.plot(ebins5[mask5], kra5_range[1] * kra5_plot ,
                 c =colors[0], ls='-', zorder=10, lw=2, label='KRA$_\gamma^{5}$ Best-Fit Flux')
        ax.fill_between(ebins5[mask5], kra5_range[0] * kra5_plot , kra5_range[-1] * kra5_plot,
                color = colors[0], alpha=.25, zorder=10)

        kra50_plot = ebins50[mask50]**2 *(kra50_flux[mask50]) * hp.nside2pixarea(128) / 3 / (sky_area)
        ax.plot(ebins50[mask50], kra50_range[1] * kra50_plot,
                 c =colors[2], ls='-', zorder=5, lw=2, label='KRA$_\gamma^{50}$ Best-Fit Flux')
        ax.fill_between(ebins50[mask50],kra50_range[0] * kra50_plot, kra50_range[-1] * kra50_plot ,
                color = colors[2], zorder=5,  alpha=.25)

        ax.plot(10**georg_energy[maskpi0_georg], pi0_range[1] * georg_flux[maskpi0_georg] * (4*np.pi / sky_area) * pi0_flux_ratio,
                lw=2, c=colors[1], ls='-', zorder=3,
                label='$\pi^0$ Best-Fit Flux')
        ax.fill_between(
            10**georg_energy[maskpi0_georg],
            pi0_range[0] * georg_flux[maskpi0_georg] * (4*np.pi / sky_area) * pi0_flux_ratio,
            pi0_range[-1] * georg_flux[maskpi0_georg] * (4*np.pi / sky_area) * pi0_flux_ratio,
            zorder=3, color=colors[1], alpha=.25
        )


    # plot Diffuse
    if True:
        #plt.plot(Es, 4*np.pi*bestfithans, c='dimgrey', lw='1', alpha=0.4, ls='--', label='M.G.Aartsen(2020)')
        plt.fill_between(
            xs, mins, maxs, color='dimgrey', alpha=0.1, edgecolor='k',
            label=r'M.G.$\,$Aartsen et al.(2020)',
            #label=r'M.G.$\,$Aartsen et al.(2020) [$E_\nu$ per-flavor]',
        )

    #ax.plot(10**georg_energy, georg_flux_gamma * (4*np.pi / sky_area) * pi0_flux_ratio, color=colors[1],
    #        lw=2, alpha=0.8, ls='-.', label=r'$\pi^0$ Model [$E_\gamma$]')
    ax.plot(10**georg_energy, georg_flux * (4*np.pi / sky_area) * pi0_flux_ratio, color=colors[1],
            lw=2, alpha=0.8, ls=':',
            #label=r'$\pi^0$ Model [$E_\nu$ per-flavor]',
            label=r'$\pi^0$ Model',
           )
    ax.plot(ebins5, ebins5**2 * (kra5_flux) * hp.nside2pixarea(128) / 3 / (sky_area), ls=':', color=colors[0] ,
            lw=2, alpha=0.8,
            #label=r'KRA$_\gamma^{5}$ Model [$E_\nu$ per-flavor]',
            label=r'KRA$_\gamma^{5}$ Model',
           )

    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylabel(r'E$_{\gamma}^2$ $\frac{dN}{dE_\nu}$ [GeV s$^{-1}$ cm$^{-2}$ sr$^{-1}$]')
    ax.set_xlabel(r'E [GeV]')
    ax.set_ylim(1e-9,1.5e-6)
    ax.set_xlim(500, 1e7)
    ax.legend(ncol=2)
    ax.set_title(
        '{}'.format(lon_min_deg) + r'$^\circ < l < $' + '{}'.format(lon_max_deg) +
        r'$^\circ$, $|b| < $' + '{}'.format(lat_deg) + r'$^\circ$')
    fig.savefig('{}/gp_flux_comparison_l{:03d}_{:03d}_b{:02d}.png'.format(
        plot_dir, lon_min_deg, lon_max_deg, lat_deg))

#### Scratch Area

In [ ]:
def powerlaw(x, gamma=-2.7):
    return x**gamma

e_primary = np.logspace(2, 7, 100)
e_gamma = e_primary * 0.1
e_nu = e_primary * 0.05
dNdE_primary = powerlaw(e_primary)

fig, ax = plt.subplots()
ax.plot(e_primary, dNdE_primary, label='primary', color=colors[0])
ax.plot(e_nu, dNdE_primary, label='nu', color=colors[2])
ax.plot(e_gamma, dNdE_primary, label='gamma', color=colors[1])
ax.plot(e_gamma, dNdE_primary * 0.31, label='gamma', ls='--', color=colors[1])
ax.plot(e_gamma*.5, dNdE_primary, label='gamma', ls='-.', color=colors[1])

ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
